In [1]:
import requests
import matplotlib.pyplot as plt
from IPython.display import Image
import folium
import pickle
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import pyproj

%matplotlib inline

#conda install -c conda-forge folium

## Using the API to pull building permit application data, as well as approved building permit data, that is related to commercial building.

#### First bring in the shapefiles for use later:

In [2]:
#council_districts = gpd.read_file('../data/CD/CD.shp')

In [3]:
business_improvement_districts = gpd.read_file('../data/BID/BID.shp')

In [4]:
#check for EPSG 4326
print(business_improvement_districts.crs)

epsg:4326


In [5]:
type(business_improvement_districts)

geopandas.geodataframe.GeoDataFrame

## Now, permit application data:
https://data.nashville.gov/Licenses-Permits/Building-Permit-Applications/kqff-rxj8

In [6]:
endpoint = 'https://data.nashville.gov/resource/kqff-rxj8.json'

In [7]:
params = {'$select': 'council_district, permit_subtype_description, mapped_location, date_entered',
    '$q': 'office, commercial', 
    '$limit': 50000}
response = requests.get(endpoint, params = params)
response.status_code

200

In [8]:
applications = response.json()
applications = pd.DataFrame(applications)
print(applications.shape)
applications.head(3)

(86, 4)


,council_district,permit_subtype_description,mapped_location,date_entered
0,33,"General Office, Professional Services","{'latitude': '36.0251', 'longitude': '-86.6159...",2019-11-08T00:00:00.000
1,18,Demolition Permit - Commercial,"{'latitude': '36.141577', 'longitude': '-86.81...",2020-08-12T00:00:00.000
2,19,"General Office, Professional Services","{'latitude': '36.155245', 'longitude': '-86.76...",2018-04-13T00:00:00.000


#### Rename headers, slice out the lat and lon as well as year:

In [9]:
applications.columns = ['Council District', 'Permit Description', 'Location', 'Year']

In [10]:
applications.dropna()

,Council District,Permit Description,Location,Year
0,33,"General Office, Professional Services","{'latitude': '36.0251', 'longitude': '-86.6159...",2019-11-08T00:00:00.000
1,18,Demolition Permit - Commercial,"{'latitude': '36.141577', 'longitude': '-86.81...",2020-08-12T00:00:00.000
2,19,"General Office, Professional Services","{'latitude': '36.155245', 'longitude': '-86.76...",2018-04-13T00:00:00.000
3,21,"Leasing / Sales Office, Other","{'latitude': '36.150553', 'longitude': '-86.80...",2019-09-06T00:00:00.000
4,14,"Medical Office, Professional Services","{'latitude': '36.173402', 'longitude': '-86.60...",2019-06-04T00:00:00.000
...,...,...,...,...
81,3,"General Office, Professional Services","{'latitude': '36.247745', 'longitude': '-86.75...",2019-01-23T00:00:00.000
82,11,"General Office, Professional Services","{'latitude': '36.226417', 'longitude': '-86.62...",2019-02-21T00:00:00.000
83,14,"General Office, Professional Services","{'latitude': '36.186123', 'longitude': '-86.62...",2020-12-29T00:00:00.000
84,19,"General Office, Civic Administration","{'latitude': '36.164679', 'longitude': '-86.77...",2020-12-18T00:00:00.000


In [11]:
applications.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86 entries, 0 to 85
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Council District    86 non-null     object
 1   Permit Description  86 non-null     object
 2   Location            86 non-null     object
 3   Year                86 non-null     object
dtypes: object(4)
memory usage: 2.8+ KB


In [12]:
#bring out year
applications['Year'] = applications['Year'].str.slice(0, 4)

In [13]:
type(applications['Location'][0])

dict

In [14]:
applications['Location'].head().apply(lambda x: x['latitude'])

0      36.0251
1    36.141577
2    36.155245
3    36.150553
4    36.173402
Name: Location, dtype: object

In [15]:
no_lat = []
for ind, val in applications['Location'].iteritems():
    try:
        val['latitude']
    except:
        print(ind, val)
        no_lat.append(ind)

23 {'human_address': '{"address": "2275 MURFREESBORO PIKE 200", "city": "ANTIOCH", "state": "TN", "zip": "37013"}'}
28 {'human_address': '{"address": "413 PORTSDALE DR", "city": "BRENTWOOD", "state": "TN", "zip": "37027"}'}
34 {'human_address': '{"address": "330 23RD AVE N 200", "city": "NASHVILLE", "state": "TN", "zip": "37203"}'}
35 {'human_address': '{"address": "3809 OCTOBER WOODS DR 102", "city": "ANTIOCH", "state": "TN", "zip": "37013"}'}
36 {'human_address': '{"address": "924B2 S DOUGLAS AVE", "city": "NASHVILLE", "state": "TN", "zip": "37204"}'}
55 {'human_address': '{"address": "28 WHITE BRIDGE PIKE 207", "city": "NASHVILLE", "state": "TN", "zip": "37205"}'}
61 {'human_address': '{"address": "1124C OLD HICKORY BLVD", "city": "MADISON", "state": "TN", "zip": "37115"}'}


In [16]:
no_lat

[23, 28, 34, 35, 36, 55, 61]

In [17]:
applications = applications.drop(no_lat)

In [18]:
applications['lat'] = applications['Location'].apply(lambda x: x['latitude'])

In [19]:
applications['lon'] = applications['Location'].apply(lambda x: x['longitude'])

In [20]:
applications.head(3)

,Council District,Permit Description,Location,Year,lat,lon
0,33,"General Office, Professional Services","{'latitude': '36.0251', 'longitude': '-86.6159...",2019,36.0251,-86.615902
1,18,Demolition Permit - Commercial,"{'latitude': '36.141577', 'longitude': '-86.81...",2020,36.141577,-86.813823
2,19,"General Office, Professional Services","{'latitude': '36.155245', 'longitude': '-86.76...",2018,36.155245,-86.768517


In [21]:
applications = applications.drop(columns = ['Location'])

#### Create a GeoDataFrame:

In [22]:
applications['geometry'] = applications.apply(lambda x: Point((float(x.lon), 
                                                         float(x.lat))), axis=1)
applications.head(3)

,Council District,Permit Description,Year,lat,lon,geometry
0,33,"General Office, Professional Services",2019,36.0251,-86.615902,POINT (-86.61590200000001 36.0251)
1,18,Demolition Permit - Commercial,2020,36.141577,-86.813823,POINT (-86.813823 36.141577)
2,19,"General Office, Professional Services",2018,36.155245,-86.768517,POINT (-86.768517 36.155245)


In [23]:
applications_geo = gpd.GeoDataFrame(applications,
                                   crs = business_improvement_districts.crs,
                                   geometry = applications['geometry'])

In [24]:
type(applications_geo)

geopandas.geodataframe.GeoDataFrame

##### Limit to CBID areas:

In [25]:
applications_geo_cbid = gpd.sjoin(business_improvement_districts, applications_geo, how = 'inner')

## Now approved permit data: 
https://data.nashville.gov/Licenses-Permits/Building-Permits-Issued/3h5w-q8b7

In [26]:
endpoint = 'https://data.nashville.gov/resource/3h5w-q8b7.json'

In [27]:
params = {'$select': 'council_dist, permit_subtype_description, mapped_location, date_issued',
    '$q': 'office, commercial', 
    '$limit': 50000}
response = requests.get(endpoint, params = params)
response.status_code

200

In [28]:
issued = response.json()
issued = pd.DataFrame(issued)
print(issued.shape)
issued.head(3)

(1881, 4)


,council_dist,permit_subtype_description,mapped_location,date_issued
0,26,"General Office, Professional Services","{'latitude': '36.088419', 'longitude': '-86.74...",2019-09-18T00:00:00.000
1,25,"General Office, Professional Services","{'human_address': '{""address"": ""3796 BEDFORD A...",2019-03-21T00:00:00.000
2,6,"General Office, Professional Services","{'latitude': '36.173405', 'longitude': '-86.76...",2019-06-25T00:00:00.000


In [29]:
issued.columns = ['Council District', 'Permit Description', 'Location', 'Year']

In [30]:
issued.dropna()

,Council District,Permit Description,Location,Year
0,26,"General Office, Professional Services","{'latitude': '36.088419', 'longitude': '-86.74...",2019-09-18T00:00:00.000
1,25,"General Office, Professional Services","{'human_address': '{""address"": ""3796 BEDFORD A...",2019-03-21T00:00:00.000
2,6,"General Office, Professional Services","{'latitude': '36.173405', 'longitude': '-86.76...",2019-06-25T00:00:00.000
3,21,"Medical Office, Professional Services","{'latitude': '36.154683', 'longitude': '-86.80...",2019-06-03T00:00:00.000
4,4,"General Office, Civic Administration","{'latitude': '36.040754', 'longitude': '-86.76...",2020-07-06T00:00:00.000
...,...,...,...,...
1876,15,"General Office, Professional Services","{'latitude': '36.146759', 'longitude': '-86.69...",2021-02-11T00:00:00.000
1877,17,"General Office, Professional Services","{'latitude': '36.11911', 'longitude': '-86.755...",2019-09-27T00:00:00.000
1878,19,"General Office, Professional Services","{'latitude': '36.150413', 'longitude': '-86.79...",2021-02-12T00:00:00.000
1879,15,"General Office, Professional Services","{'latitude': '36.148792', 'longitude': '-86.68...",2021-02-12T00:00:00.000


In [31]:
issued.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1881 entries, 0 to 1880
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Council District    1872 non-null   object
 1   Permit Description  1881 non-null   object
 2   Location            1881 non-null   object
 3   Year                1881 non-null   object
dtypes: object(4)
memory usage: 58.9+ KB


In [32]:
#bring out year
issued['Year'] = issued['Year'].str.slice(0, 4)

In [33]:
type(issued['Location'][0])

dict

In [34]:
no_lat = []
for ind, val in issued['Location'].iteritems():
    try:
        val['latitude']
    except:
        #print(ind, val)
        no_lat.append(ind)

In [35]:
issued = issued.drop(no_lat)

In [36]:
issued['lat'] = issued['Location'].apply(lambda x: x['latitude'])
issued['lon'] = issued['Location'].apply(lambda x: x['longitude'])

In [37]:
issued = issued.drop(columns = ['Location'])

#### Make a GeoDataFrame for Points

In [38]:
issued['geometry'] = issued.apply(lambda x: Point((float(x.lon), 
                                                         float(x.lat))), axis=1)
issued.head(3)

,Council District,Permit Description,Year,lat,lon,geometry
0,26,"General Office, Professional Services",2019,36.088419,-86.748409,POINT (-86.748409 36.088419)
2,6,"General Office, Professional Services",2019,36.173405,-86.763282,POINT (-86.763282 36.173405)
3,21,"Medical Office, Professional Services",2019,36.154683,-86.804336,POINT (-86.80433600000001 36.154683)


In [39]:
issued_geo = gpd.GeoDataFrame(issued,
                                   crs = business_improvement_districts.crs,
                                   geometry = issued['geometry'])

In [40]:
type(issued_geo)

geopandas.geodataframe.GeoDataFrame

##### Limit to CBID areas:

In [41]:
issued_geo_cbid = gpd.sjoin(business_improvement_districts, issued_geo, how = 'inner')

In [42]:
issued_geo_cbid.head()

,area,date_est_d,time_est_d,id,date_modif,time_modif,name,objectid,ordinance,shape_star,shape_stle,geometry,index_right,Council District,Permit Description,Year,lat,lon
0,0.598798,1998-02-06,00:00:00.000,0.0,2017-03-08,00:00:00.000,Central Business Improvement District,2.0,BL2017-580,1.669347e+07,20206.601116,"POLYGON ((-86.78557 36.15742, -86.78417 36.158...",1274,19,"Leasing / Sales Office, Other",2019,36.155686,-86.775281
0,0.598798,1998-02-06,00:00:00.000,0.0,2017-03-08,00:00:00.000,Central Business Improvement District,2.0,BL2017-580,1.669347e+07,20206.601116,"POLYGON ((-86.78557 36.15742, -86.78417 36.158...",192,19,Hotel / Motel,2019,36.156512,-86.771695
0,0.598798,1998-02-06,00:00:00.000,0.0,2017-03-08,00:00:00.000,Central Business Improvement District,2.0,BL2017-580,1.669347e+07,20206.601116,"POLYGON ((-86.78557 36.15742, -86.78417 36.158...",1554,19,Hotel / Motel,2019,36.156542,-86.775863
0,0.598798,1998-02-06,00:00:00.000,0.0,2017-03-08,00:00:00.000,Central Business Improvement District,2.0,BL2017-580,1.669347e+07,20206.601116,"POLYGON ((-86.78557 36.15742, -86.78417 36.158...",1488,19,"Leasing / Sales Office, Other",2019,36.158069,-86.780135
0,0.598798,1998-02-06,00:00:00.000,0.0,2017-03-08,00:00:00.000,Central Business Improvement District,2.0,BL2017-580,1.669347e+07,20206.601116,"POLYGON ((-86.78557 36.15742, -86.78417 36.158...",450,19,"General Office, Professional Services",2019,36.158821,-86.775739


## Bring this all into Folium:

In [43]:
#basemap (used state cap. for the lat/long)
base = folium.Map(location = [36.159996, -86.778281], zoom_start = 14.2, control_scale = True)

            
#create cluster with applications
marker_cluster = MarkerCluster(name = 'Permit Applications').add_to(base)

for row_index, row_values in applications_geo_cbid.iterrows():
        loc = row_values['lat'], row_values['lon']
        pop = str(row_values['Permit Description'])
        icon = folium.Icon(color = 'red', prefix = 'fa')
        marker = folium.Marker(location = loc,
                              popup = pop,
                              icon = icon)
        marker.add_to(marker_cluster)
        
#create cluster with issued
marker_cluster = MarkerCluster(name = 'Issued Permits').add_to(base)

for row_index, row_values in issued_geo_cbid.iterrows():
        loc = row_values['lat'], row_values['lon']
        pop = str(row_values['Permit Description'])
        icon = folium.Icon(color = 'green', prefix = 'fa')
        marker = folium.Marker(location = loc,
                              popup = pop,
                              icon = icon)
        marker.add_to(marker_cluster)
        
#add business improvement district polygons
folium.GeoJson(business_improvement_districts, name = 'Business Improvement Districts').add_to(base)

#enable layer control
folium.LayerControl().add_to(base)

In [44]:
base

### Takeaways from folium map:

###### Note that red markers indicate permit applications, where green indicate issued permits.